In [1]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
import numpy as np

In [2]:
#data = pd.read_csv("/content/tweet_emotions.csv")
#data = pd.read_csv("sample.csv")
data = pd.read_csv("text.csv")

In [3]:
#data.head()

In [ ]:
# emotions = data["sentiment"].unique()
# emotion_id = {emotion: i for i, emotion in enumerate(emotions)}

# data["emotion_id"] = data["sentiment"].map(emotion_id)

In [4]:
mapping = {0 : "sadness",
           1 : "joy",
           2 : "love",
           3 : "anger",
           4 : "fear",
           5 : "surprise"
}

data["emotion"] =  data["label"].map(mapping)

In [5]:
#data.head()

,Unnamed: 0,text,label,emotion
0,0,i just feel really helpless and heavy hearted,4.0,fear
1,1,ive enjoyed being able to slouch about relax a...,0.0,sadness
2,2,i gave up my internship with the dmrg and am f...,4.0,fear
3,3,i dont know i feel so lost,0.0,sadness
4,4,i am a kindergarten teacher and i am thoroughl...,4.0,fear


Creating the evaluation dataset

In [5]:

np.random.seed(42)
data = data.sample(frac=1).reset_index(drop=True)

eval_ratio = 0.15
total = data.shape[0]
eval_end = int(total * eval_ratio)

eval_data = data[:eval_end]

data = data[eval_end:]
print(total, len(eval_data), len(data), len(eval_data) + len(data))

338471 50770 287701 338471


Setting up the model


In [6]:
model_nm = "FacebookAI/roberta-base"

In [8]:
#! pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 11.6 MB/s eta 0:00:00


In [7]:
from datasets import Dataset, DatasetDict
ds = Dataset.from_pandas(data)
# ds_eval = Dataset.from_pandas(eval_data)
# ds_dict = DatasetDict({"train": ds, "eval": ds_eval})

Tokenizer

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
tokz = AutoTokenizer.from_pretrained(model_nm)

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
def tok_func(x): return tokz(x["text"])

In [10]:
tokz_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/287701 [00:00<?, ? examples/s]

In [ ]:
#eval_tokz_ds = ds.map(tok_func, batched=True)

In [11]:
eval_ds = Dataset.from_pandas(eval_data).map(tok_func, batched=True)

Map:   0%|          | 0/50770 [00:00<?, ? examples/s]

In [14]:
t#okz_ds[0]

{'Unnamed: 0': 15888,
 'text': 'i feel like he may be ashamed of me',
 'label': 0.0,
 'emotion': 'sadness',
 'input_ids': [0, 118, 619, 101, 37, 189, 28, 20085, 9, 162, 2],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Train-Test split


In [12]:
dds = tokz_ds.train_test_split(test_size=0.2)
dds

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'label', 'emotion', 'input_ids', 'attention_mask'],
        num_rows: 230160
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'label', 'emotion', 'input_ids', 'attention_mask'],
        num_rows: 57541
    })
})

Args

In [13]:
bs = 96
lr = 8e-5
epochs = 1

In [14]:
from transformers import TrainingArguments, Trainer

In [ ]:
#!pip install accelerate -U

In [ ]:
#!pip install -U transformers

In [ ]:
# import accelerate
# import transformers
# accelerate.__version__, transformers.__version__

In [19]:
#!pip install transformers==4.17
#--- solved the error of the install accelerate!
#torch default version is not compatible with the accelerate version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 24.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1


In [15]:
args = TrainingArguments(
    "outputs",
    learning_rate=lr,
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs,
    warmup_ratio=0.1,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    fp16=True,
    evaluation_strategy="epoch",
    report_to="none",
    save_total_limit=3
)

Metrics

In [16]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from transformers import EvalPrediction
import torch
from torch.nn.functional import cross_entropy

def multi_label_metrics(predictions, labels):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.argmax(probs, axis=1)
    y_true = labels
    accuracy = accuracy_score(y_true, y_pred)
    #loss = cross_entropy(torch.Tensor(predictions), torch.Tensor(labels))
    metrics ={"accuracy":accuracy}
    return metrics


def compute_metrics(p:EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(predictions= preds, labels = p[1])
    print(result)
    return result

In [17]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=6)
trainer = Trainer(
    model,
    args,
    train_dataset=dds["train"],
    eval_dataset=dds["test"],
    tokenizer=tokz,
    compute_metrics=compute_metrics
)

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at FacebookAI/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.out

In [18]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: emotion, text, Unnamed: 0. If emotion, text, Unnamed: 0 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 230160
  Num Epochs = 1
  Instantaneous batch size per device = 96
  Total train batch size (w. parallel, distributed & accumulation) = 96
  Gradient Accumulation steps = 1
  Total optimization steps = 2398


Epoch,Training Loss,Validation Loss,Accuracy
1,0.093000,0.090753,0.940929


Saving model checkpoint to outputs/checkpoint-500
Configuration saved in outputs/checkpoint-500/config.json
Model weights saved in outputs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in outputs/checkpoint-500/tokenizer_config.json
Special tokens file saved in outputs/checkpoint-500/special_tokens_map.json
Saving model checkpoint to outputs/checkpoint-1000
Configuration saved in outputs/checkpoint-1000/config.json
Model weights saved in outputs/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in outputs/checkpoint-1000/tokenizer_config.json
Special tokens file saved in outputs/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to outputs/checkpoint-1500
Configuration saved in outputs/checkpoint-1500/config.json
Model weights saved in outputs/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in outputs/checkpoint-1500/tokenizer_config.json
Special tokens file saved in outputs/checkpoint-1500/special_tokens_map.json
Saving model check

{'accuracy': 0.9409290766583828}




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2398, training_loss=0.18756315566978424, metrics={'train_runtime': 817.1731, 'train_samples_per_second': 281.654, 'train_steps_per_second': 2.935, 'total_flos': 6935478271645824.0, 'train_loss': 0.18756315566978424, 'epoch': 1.0})

In [19]:
trainer.save_model("BERT_Emotions")


Saving model checkpoint to BERT_Emotions
Configuration saved in BERT_Emotions/config.json
Model weights saved in BERT_Emotions/pytorch_model.bin
tokenizer config file saved in BERT_Emotions/tokenizer_config.json
Special tokens file saved in BERT_Emotions/special_tokens_map.json


In [20]:
preds = trainer.predict(eval_ds).predictions.astype(float)
print(multi_label_metrics(preds, eval_ds['label']))

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: emotion, text, Unnamed: 0. If emotion, text, Unnamed: 0 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 50770
  Batch size = 192


{'accuracy': 0.9419145164467205}
{'accuracy': 0.9419145164467205}


In [ ]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)